# **Pandas Data Transformations**

In [123]:
# group by and aggregations:: Data transformations is bridge between raw data and actionable insights.
import pandas as pd
penguins_body_mass=pd.read_csv("..\Datasets\penguins.csv")

# Aggregations in pandas

Aggregation operations summarize data within groups, reducing multiple values to a single value per group. They are a cornerstone of the split-apply-combine strategy in pandas' groupby:

- Split: Break the DataFrame into groups (e.g., by `body_mass_g`).
- Apply: Compute a summary (e.g., sum, mean, count) for each group.
- Combine: Collect results into a new DataFrame or Series.

Basic aggregation functions include:
- Numeric: `mean()`, `sum()`, `min()`, `max()`, `std()`, `median()`.
- General: `count()` (counts non-NaN values), `nunique()` (counts unique values), `size()` (counts total rows including NaNs).
- Custom: Use `.agg()` with functions or lambdas, e.g., `.agg(lambda x: max(x) - min(x))` for range, or `.agg(list)` to collect values.

Aggregations combine multiple values into a single result for each group, enabling efficient high-level insights into the data.


In [124]:
# Group data by 'species' and calculate the mean body mass for each group.
# This groups the penguins by species, then finds the average weight in grams.

penguins_body_mass.groupby(["species"])["body_mass_g"].mean()

species
Adelie       3700.662252
Chinstrap    3733.088235
Gentoo       5076.016260
Name: body_mass_g, dtype: float64

The code :
penguins_body_mass.groupby(["body_mass_g"])["species"].mean()

did not work because the `.mean()` function in pandas requires numeric data and fails on strings.

- `.mean()` → works only on numeric data, so it raises an error or returns nothing when applied to non-numeric columns like strings (e.g., `"species"`).
- `.sum()` → with strings, it concatenates the values instead of summing numerically.
- `.min()` / `.max()` → applied to strings, these return the lexicographically smallest or largest value respectively.

Thus, `.mean()` is stricter and numeric-only, whereas `.sum()`, `.min()`, and `.max()` can operate on string data but with behavior that reflects string operations rather than arithmetic.

In [125]:
# Get unique species values (3 unique species in dataset)
penguins_body_mass["species"].unique()
# three unique values. 

array(['Adelie', 'Gentoo', 'Chinstrap'], dtype=object)

In [126]:
# Using agg() to compute multiple aggregation metrics for each species
penguins_body_mass.groupby("species")["flipper_length_mm"].agg(["sum","mean"])

,sum,mean
species,,
Adelie,28683.0,189.953642
Chinstrap,13316.0,195.823529
Gentoo,26714.0,217.186992


In [127]:
result = penguins_body_mass.groupby("species").agg({
    "flipper_length_mm": ["sum", "mean"],   # multiple aggregations for flipper length
    "body_mass_g": ["mean", "max", "min"]   # multiple aggregations for body mass
})
result

flipper_length_mm              body_mass_g                
                        sum        mean         mean     max     min
species                                                             
Adelie              28683.0  189.953642  3700.662252  4775.0  2850.0
Chinstrap           13316.0  195.823529  3733.088235  4800.0  2700.0
Gentoo              26714.0  217.186992  5076.016260  6300.0  3950.0

In [128]:
# Count non-null body_mass_g values for each species
penguins_body_mass.groupby("species")["body_mass_g"].count()

species
Adelie       151
Chinstrap     68
Gentoo       123
Name: body_mass_g, dtype: int64

In [129]:
# Advanced aggregations: applying multiple aggregation functions to summarize data.

aggregations= penguins_body_mass.groupby("island")["year"].agg(["max","min"])
aggregations

,max,min
island,,
Biscoe,2009,2007
Dream,2009,2007
Torgersen,2009,2007


In [130]:
# To make results clearer, we can rename aggregation columns using a dictionary.

renamed_columns={
    "maximum year":"max",
    "average_year":"mean"
}
aggregations= penguins_body_mass.groupby("species")["year"].agg(**renamed_columns)
aggregations

,maximum year,average_year
species,,
Adelie,2009,2008.013158
Chinstrap,2009,2007.970588
Gentoo,2009,2008.080645


In [131]:
# Applying different aggregation functions to multiple columns.
aggregations= penguins_body_mass.groupby("island")["year"].agg(max_year="max",min_year="min")
aggregations

,max_year,min_year
island,,
Biscoe,2009,2007
Dream,2009,2007
Torgersen,2009,2007


In [132]:
# Applying different aggregation functions to multiple columns.
penguins_body_mass.groupby("island").agg({"flipper_length_mm":"min", "bill_length_mm":"min"})

,flipper_length_mm,bill_length_mm
island,,
Biscoe,172.0,34.5
Dream,178.0,32.1
Torgersen,176.0,33.5


In [133]:
# Combine multiple aggregations into one grouped summary.
penguins_body_mass.groupby("island").agg(
    average_body_mass=("body_mass_g", "mean"),
    unique_species_count=("species", "nunique"),
    std_body_mass=("body_mass_g", "std") #standard deviation 
)

,average_body_mass,unique_species_count,std_body_mass
island,,,
Biscoe,4716.017964,2,782.855743
Dream,3712.903226,2,416.644112
Torgersen,3706.372549,1,445.107940


- **Aggregations** such as `sum()`, `mean()`, `min()`, `max()`, `count()`, and `nunique()` summarize data within groups.
- **Multiple aggregations** can be applied simultaneously using `.agg()` with lists, dictionaries, or named aggregation syntax.
- **Named aggregations** allow assigning descriptive names to output columns for clarity.
- Proper data types (numeric for math functions) are crucial to avoid errors during aggregation.
- For categorical data, functions like `nunique` or `count` provide meaningful insights rather than numeric aggregates like `sum`.

## Best Practices
- Use **named aggregations** for clean, readable output:
- When aggregating strings, avoid numeric functions; use counts or unique counts instead.
- Rename columns either during aggregation (named aggregation) or afterwards using `.rename()`.
- Examine and convert data types before aggregation to prevent errors like `TypeError: agg function failed`.

## Summary Table of Useful Aggregations

| Function      | Description                             | Use Case                |
|---------------|-------------------------------------|-------------------------|
| `sum()`       | Sum of values                       | Numeric totals          |
| `mean()`      | Arithmetic average                  | Central tendency        |
| `min()`/`max()`| Minimum/Maximum values              | Range insights          |
| `count()`     | Non-null count                     | Number of entries       |
| `nunique()`   | Count of unique values             | Category diversity      |
| `std()`       | Standard deviation                 | Data variability        |

Mastering `groupby` and `agg` empowers efficient exploratory data analysis and reporting by summarizing complex datasets into comprehensible insights. Always choose aggregation functions considering data types and the analysis context for meaningful results.

In [134]:
# using custom Aggregations functions: custom functions helps us to do advanced action tailored to our specific names.
import pandas as pd
penguins_data=pd.read_csv("../Datasets/penguins.csv")

## why custom Aggregations functions :?
- When built-in functions doesnot meet our requirements
- To implement specific business rules for analysis. 
-  Advanced metrices such as ranges, percentages are required.

### Key Methods

- **Basic Syntax:**  
  ```
  df.groupby('column')['column_to_aggregate'].agg(your_function)
  ```

- **Lambda:**  
  ```
  agg(lambda x: max(x) - min(x))
  ```

- **Named Function:**  
  ```
  def my_func(x):
      return ...
  agg(my_func)
  ```

- **Multiple Functions:**  
  ```
  agg([my_func, 'mean'])
  # or
  agg({'column': [(name, my_func)]})  # for named outputs
  ```
```


In [135]:
def calculate_range(series):
    return series.max()-series.min()
custom_aggregations = penguins_data.groupby("species")["body_mass_g"].agg(
    unique_species= "nunique",
    mass_min="min",
    mass_max="max",
    mass_range=calculate_range 
)
custom_aggregations

# The column used for aggregation is determined by what you select after the groupby (here body_mass_g).

# The aggregation functions (like min, max) are applied to this selected column group-wise.

# The output column names inside .agg() come from the keyword argument names you provide.

,unique_species,mass_min,mass_max,mass_range
species,,,,
Adelie,55,2850.0,4775.0,1925.0
Chinstrap,34,2700.0,4800.0,2100.0
Gentoo,47,3950.0,6300.0,2350.0


In [136]:
custom_agg = penguins_data.groupby('island').agg(
    unique_species=('species', 'nunique'),
    mass_range=('body_mass_g', lambda s: s.max() - s.min())
)
custom_agg

# Groups penguins by island.
# Counts unique species in each island group.
# Calculates range (max - min) of body_mass_g per island.

,unique_species,mass_range
island,,
Biscoe,2,3450.0
Dream,2,2100.0
Torgersen,1,1800.0


In [137]:
# Custom function: count values above 4000
def count_heavy(x):
    return (x > 4000).sum()

result = penguins_body_mass.groupby('species')['body_mass_g'].agg(count_heavy)
print(result)

species
Adelie        35
Chinstrap     15
Gentoo       122
Name: body_mass_g, dtype: int64


In [138]:
# custom complex aggregarions
penguins_adelie=penguins_body_mass.query("species=='Adelie'")
len(penguins_adelie["island"])

152

In [139]:
sum(penguins_adelie["island"]=="Biscoe")
# .sum() on this mask counts the number of True values (matches), while len() just returns total length unrelated to matching count.

44

To calculate the percentage of each species within a pandas DataFrame using custom complex aggregations, you can apply a groupby operation combined with a custom aggregation function. The general approach is:

1. Group the DataFrame by the categorical column (`species`).
2. Count the number of entries in each group.
3. Divide each group's count by the total count of all entries (to get the proportion).
4. Multiply by 100 to convert the proportion to a percentage.

Here is an example code snippet implementing this:

In [ ]:
def percentage_of_total(x):
    return (len(x) / len(penguins_body_mass)) * 100
result = penguins_body_mass.groupby("species").agg(percentage=('species', percentage_of_total))
result

,percentage
species,
Adelie,44.186047
Chinstrap,19.767442
Gentoo,36.046512


In [148]:
result=penguins_body_mass.groupby("island").agg(percentage=("island",percentage_of_total))
result

,percentage
island,
Biscoe,48.837209
Dream,36.046512
Torgersen,15.116279


In [151]:
def percentage_of_total(x):
    return (len(x) / len(penguins_body_mass)) * 100

# Group by both 'island' and 'species',
# then apply the function to count rows per group and convert to percentage of total
result = penguins_body_mass.groupby(['island', 'species']).agg(percentage=('species', percentage_of_total))
result.reset_index()


,island,species,percentage
0,Biscoe,Adelie,12.790698
1,Biscoe,Gentoo,36.046512
2,Dream,Adelie,16.279070
3,Dream,Chinstrap,19.767442
4,Torgersen,Adelie,15.116279
